In [1]:
from crossreference import *
import chart.johncartersqueeze as jcsqueeze

In [2]:
import pandas as pd
import numpy as np
import datetime

In [3]:
import mysql.connector

def load_stockdata(symbol):
    serverName = "localhost"
    cnx = mysql.connector.connect(user='michael', password='Password2017',
                                  host=serverName, database='stock_market')
    cursor = cnx.cursor()
    
    query = \
    "SELECT stock.*, bb.`sma20` AS bb_middle, bb.`band_upper` AS bb_upper, bb.`band_lower` AS bb_lower, \
    kc.`ema20` AS kc_middle, kc.`atr2_upper` AS kc_2_upper, kc.`atr2_lower` AS kc_2_lower, \
    kc.`atr3_upper` AS kc_3_upper, kc.`atr3_lower` AS kc_3_lower, \
    kc.`atr1_upper` AS kc_1_upper, kc.`atr1_lower` AS kc_1_lower, \
    macd.`macd_histogram` as macd, ttma.macd_histogram as wave_a, ttmb.macd_histogram as wave_b, ttmc.macd_histogram as wave_c \
    FROM \
    ( SELECT * FROM (SELECT symbol, effective_date, `adj_open`, `adj_high`, `adj_low`, `adj_close`, `adj_volume` FROM `market.stock_price` \
      WHERE symbol = '{0}' ORDER BY effective_date DESC LIMIT 180) sub ORDER BY `effective_date` ASC ) stock \
    LEFT JOIN \
    `idc.bollinger_bands` AS bb ON bb.symbol = stock.symbol AND bb.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.keltner_channels` AS kc ON kc.symbol = stock.symbol AND kc.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.macd` as macd on macd.symbol = stock.symbol and macd.effective_date = stock.effective_date \
        and macd.short_period = 12 and macd.long_period=26 and macd.signal_period=9 \
    LEFT JOIN \
    `idc.macd` as ttma on ttma.symbol = stock.symbol and ttma.effective_date = stock.effective_date \
        and ttma.short_period = 8 and ttma.long_period=34 and ttma.signal_period=34 \
    LEFT JOIN \
    `idc.macd` as ttmb on ttmb.symbol = stock.symbol and ttmb.effective_date = stock.effective_date \
        and ttmb.short_period = 8 and ttmb.long_period=89 and ttmb.signal_period=89 \
    LEFT JOIN \
    `idc.macd` as ttmc on ttmc.symbol = stock.symbol and ttmc.effective_date = stock.effective_date \
        and ttmc.short_period = 8 and ttmc.long_period=144 and ttmc.signal_period=144 \
    ".format(symbol)

    cursor.execute(query)
    dfsql = pd.DataFrame(cursor.fetchall(), dtype=np.float)
    dfsql.columns = cursor.column_names
    dfsql.set_index('effective_date', inplace=True)
    cursor.close()
    return dfsql


In [4]:
df = load_stockdata('AMZN')
df['macd_up'] = df.macd > df.macd.shift(-1)
df.head(20)

,symbol,adj_open,adj_high,adj_low,adj_close,adj_volume,bb_middle,bb_upper,bb_lower,kc_middle,...,kc_2_lower,kc_3_upper,kc_3_lower,kc_1_upper,kc_1_lower,macd,wave_a,wave_b,wave_c,macd_up
effective_date,,,,,,,,,,,,,,,,,,,,,
2018-02-15,AMZN,1466.89,1468.94,1436.84,1461.76,5598111.0,1395.4670,1485.223579,1305.710421,1384.004065,...,1281.225959,1538.171224,1229.836906,1435.393118,1332.615012,-2.381587,2.410285,69.318967,122.784881,True
2018-02-14,AMZN,1406.25,1452.06,1403.36,1451.05,5881238.0,1387.0450,1481.809828,1292.280172,1375.819230,...,1270.073578,1534.437708,1217.200752,1428.692056,1322.946404,-5.229985,-2.836118,63.642456,116.178967,True
2018-02-13,AMZN,1385.93,1419.72,1383.53,1414.51,5858860.0,1379.2425,1477.283232,1281.201768,1367.900201,...,1261.512575,1527.481640,1208.318762,1421.094014,1314.706388,-8.891311,-9.157877,57.169088,108.844506,True
2018-02-12,AMZN,1364.67,1393.81,1344.01,1386.23,6682214.0,1373.7600,1475.492717,1272.027283,1362.993907,...,1253.990311,1526.499301,1199.488513,1417.495705,1308.492109,-10.932459,-10.974546,56.514806,107.632653,True
2018-02-09,AMZN,1373.49,1383.50,1265.93,1338.99,13787886.0,1369.7085,1475.504916,1263.912084,1360.548002,...,1251.593360,1523.979965,1197.116039,1415.025323,1306.070681,-11.091325,-8.073099,61.531242,112.286054,False
2018-02-08,AMZN,1429.67,1433.75,1349.94,1350.47,8108560.0,1366.5930,1479.270378,1253.915622,1362.817266,...,1263.569190,1511.689380,1213.945152,1412.441304,1313.193228,-6.343089,5.794422,79.099867,129.812532,False
2018-02-07,AMZN,1449.00,1460.99,1415.15,1416.78,7066828.0,1361.7860,1484.555575,1239.016425,1364.116978,...,1270.128280,1505.100025,1223.133931,1411.111327,1317.122629,-0.182016,21.749384,97.875581,148.262229,False
2018-02-06,AMZN,1361.46,1443.99,1351.79,1442.84,10773963.0,1353.5820,1482.338472,1224.825528,1358.573502,...,1264.407212,1499.822937,1217.324067,1405.656647,1311.490357,2.112128,27.758871,103.610394,152.888656,False
2018-02-05,AMZN,1402.62,1458.98,1320.72,1396.62,10756332.0,1343.7835,1473.699930,1213.867070,1349.703344,...,1262.478110,1480.541195,1218.865493,1393.315961,1306.090727,2.142406,28.231339,102.328930,150.148906,False


In [5]:
jcsqueeze.drawchart('AMZN', df)

In [3]:
import repository.stockpricerepo as pricerepo

In [5]:
df = pricerepo.get_stock_adjdata('AMZN', datetime.date(2017,1,1), datetime.date(2018,2,20))
df.head()

,adj_open,adj_high,adj_low,adj_close,adj_volume
effective_date,,,,,
2017-01-03,757.92,758.7595,747.7000,753.67,3521066.0
2017-01-04,758.39,759.6800,754.2000,757.18,2510526.0
2017-01-05,761.55,782.3999,760.2557,780.45,5830068.0
2017-01-06,782.36,799.4400,778.4800,795.99,5986234.0
2017-01-09,798.00,801.7742,791.7700,796.92,3446109.0


In [6]:
import repository.simpleindicatorrepo as pricerepo

ImportError: cannot import name 'indicatorrepo'